In [1]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()

In [2]:
base_url = "https://www.prisjakt.nu"
url = "https://www.prisjakt.nu/c/tv"
page_links = []
links = []
page_links.append(url)
for page in page_links:
    url = page
    i = requests.get(url).text
    soup=BeautifulSoup(i,'html.parser')
    pages = soup.find_all("li",{"class":"PaginationListItem-sc-11oisbu-1 bKXcXk"})
    for page in pages:
        link = page.find("a", {"data-test":"PaginationLink"})
        links.append(link)
        if link == None:
            links.remove(link)
    for l in links:
        pg = l.get('href')
        full_url = base_url+pg
        while full_url not in page_links:
            page_links.append(full_url)
        

In [3]:
len(page_links)

47

In [4]:
productlists = []
for link in page_links:
        k = requests.get(link).text
        soup=BeautifulSoup(k,'html.parser')
        productlist = soup.find_all("article",{"data-test":"ProductGridCard"})
        productlists.append(productlist)

In [5]:
items = [item for sublist in productlists for item in sublist]

In [6]:
productlinks = []
for item in items:
        link = item.find("a",{"data-test":"InternalLink"}).get('href')                 
        productlinks.append(base_url+link)

In [7]:
len(productlinks)

2043

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [9]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\user\AppData\Local\Temp\ipykernel_14176\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [13]:
data=[]
for link in productlinks[:10]:
    from datetime import datetime
    f = requests.get(link,headers=headers).text
    hun=BeautifulSoup(f,'html.parser')
    driver.get(link)
    try:
        element1 = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.BannerCookieBanner div.BannerContentCookieBanner div.ButtonsWrapperCookieBanner.AcceptButtonWrapperCookieBanner button.AcceptButtonCookieBanner.ButtonCookieBanner span.TextWrapperCookieBanner > span:nth-child(1)"))
            )
        element1.click()
    except:
        element1 = None
    try:
        element2 = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.App-sc-2fu3f8-0.ljZBoE section.Main-sc-2fu3f8-1.hnCjSa:nth-child(5) div.Content-sc-2fu3f8-2.hybPGh div.PageContent-sc-1wgu331-5.dfQdce div.Wrapper-sc-1wgu331-1.gPsMfp div.Content-sc-1wgu331-0.gMPJqu div.ProductPage-sc-mnx6sq-0.iMqgaO div.SectionWrapper-sc-ia0zhw-0.bqrwQK:nth-child(1) div.PageSectionWrapper-sc-o2pbvx-0.gHOcYt.StyledPageSection-sc-1w3nqup-0.UUqee:nth-child(1) section.PageSectionContent-sc-o2pbvx-1.euuGdp section:nth-child(1) div.PriceListPanelWrapper-sc-1htx0wh-0.lkPiWw div.PricePanel-sc-wkzg9v-1.kSsiTe div:nth-child(3) > button.BaseButton--uexw3i.cVdzxy.textbutton"))
            )
        element2.click()
    except:
        element2 = None
    
    from parsel import Selector
    def get_prop():
        driver.get(link+'#properties')
        sel = Selector(text=driver.page_source)
        parsed = []
        for item in sel.xpath("//div[contains(@class,'Column-sc-1jnk5ag-2 bIOAXd')]"):
            parsed.append({
                'title': item.css('span[class]').get(),
            })
        # split dictionary into keys and values
        keys = []
        values = []
        for item in parsed:
            for i in item:
                keys.append(i)
                values.append(item[i])
        bs4 = []
        for item in values:
            prop = BeautifulSoup(item,'html.parser')
            bs4.append(prop)
        cols = []
        vals = []
        for str in bs4:
            name = str.find("span", {"class":"Text--bzqghn wmwsE bodysmalltext PropertyName-sc-1jnk5ag-5 hEdIXF"})
            if name != None :
                cols.append(name.text.replace('\n',""))
            val = str.find("span", {"class":"Text--bzqghn ddqBq bodysmalltext PropertyValue-sc-1jnk5ag-6 etcKHO"})
            if val != None :
                vals.append(val.text.replace('\n',""))

        properties = dict(zip(cols, vals))
        return properties

    prices = []
    price_list = hun.find_all('div', class_='PriceWrapper-sc-k40pbc-4 OYetQ')
    for price in price_list:
        for p in price:
            pri = p.get_text()
            pri = re.sub('[\xa0]', '', pri)
            pri = re.sub('[kr]', '', pri)
            pri = pri.replace(r'[()]',"")
            prices.append(pri)
    prices = [int(item) for item in prices]
    

    try:
        category = "TV"
    except:
        category = None
    try:
        name = hun.find("div",{"data-test":"ProductTitle"}).text.replace('\n',"")
    except:
        name = None
    try:
        avg_price = mean(prices)
    except:
        avg_price = None
    try:
        offs = len(prices)
    except:
        offs = None
    try:
        country = "SW"
    except:
        country = None
    try:
        currency = "SEK"
    except:
        currency = None
    try:
        rank = hun.find("span",{"class":"Text--bzqghn kBBoFI titlesmalltext StyledCaptionText--7mhtyj fxXtPM"}).text.replace('\n',"")
    except:
        rank = None
    try:
        rating = hun.find("span",{"class":"RateNumber-sc-14ktvqu-5 ifhFUL"}).text.replace('\n',"")
    except:
        rating = None
    try:
        reviews = hun.find("span",{"class":"Counter-sc-14ktvqu-0 bORxLl"}).text.replace('\n',"")
    except:
        reviews = None
    try:
        description = hun.find("p",{"class":"ProductInfo-sc-46oo5c-0 bRxlhS"}).text.replace('\n',"")
    except:
        description = None
    #try:
        #specifications = get_specs(prod_specifications)
    #except:
        #specifications = None
        
    try:
        product_specs = get_prop()
    except:
        product_specs
    try:
        scrape_link = link
        
    except:
        scrape_link = None
    
    try:
        now = datetime.now()
        date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
    except:
        date_scraped = None

        
    tv = {"Country Code":country, "Category":category, "Name":name, "Avg Price": avg_price, "Currency":currency, "Nb of Offers":offs, "Rank":rank, "Rating":rating, "Reviews":reviews, "Description":description, \
        "scrape_link":scrape_link, "date_scraped":date_scraped, "Product Specification":product_specs}
    
    data.append(tv)

In [14]:
df = pd.DataFrame.from_dict(data)
df.shape

(10, 13)

In [15]:
df.columns

Index(['Country Code', 'Category', 'Name', 'Avg Price', 'Currency',
       'Nb of Offers', 'Rank', 'Rating', 'Reviews', 'Description',
       'scrape_link', 'date_scraped', 'Product Specification'],
      dtype='object')

In [17]:
spec_df = df['Product Specification'].apply(pd.Series)
df = pd.concat([df, spec_df.reindex(df.index)], axis=1)
#gen_spec_df = df["Generalspecifications"].apply(pd.Series)
#df = pd.concat([df, gen_spec_df.reindex(df.index)], axis=1)
df_c = df.copy()
# Remove specifications column
#df.drop(['specifications'], axis=1, inplace=True)
df.head()

,Country Code,Category,Name,Avg Price,Currency,Nb of Offers,Rank,Rating,Reviews,Description,...,HDR,Smart TV,Lanseringsår,Varumärke,Bildstorlek,Upplösning,Bildskärmsteknik,HDR,Smart TV,Lanseringsår
0,SW,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",17596.733333,SEK,15,Rank 1,4.2,(22 omdömen),"Det billigaste priset för LG OLED65C1 65"" 4K U...",...,Ja,Ja,2021,LG,65 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021
1,SW,TV,"LG OLED55C1 55"" 4K Ultra HD (3840x2160) OLED S...",12901.133333,SEK,15,Rank 2,4.1,(3 omdömen),"Det billigaste priset för LG OLED55C1 55"" 4K U...",...,Ja,Ja,2021,LG,55 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021
2,SW,TV,"Philips 58PUS9006 58"" 4K Ultra HD (3840x2160) ...",9675.714286,SEK,7,Rank 3,3.7,(2 omdömen),Det billigaste priset för Philips 58PUS9006 58...,...,Ja,Ja,,Philips,58 tum,4K Ultra HD (3840x2160),LCD,Ja,Ja,
3,SW,TV,"LG OLED77B1 77"" 4K Ultra HD (3840x2160) OLED S...",23323.333333,SEK,3,Rank 4,4.1,(4 omdömen),"Det billigaste priset för LG OLED77B1 77"" 4K U...",...,Ja,Ja,2021,LG,77 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021
4,SW,TV,"LG OLED55B1 55"" 4K Ultra HD (3840x2160) OLED S...",10875.500000,SEK,10,Rank 5,4.2,(5 omdömen),"Det billigaste priset för LG OLED55B1 55"" 4K U...",...,Ja,Ja,2021,LG,55 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021


In [18]:
spec_df.columns

Index(['Varumärke', 'Bildstorlek', 'Upplösning', 'Bildskärmsteknik', 'HDR',
       'Smart TV', 'Lanseringsår'],
      dtype='object')

In [13]:
df.drop(['Varumärke'], axis=1, inplace=True)

In [14]:
df = pd.concat([df, spec_df['Varumärke'].reindex(df.index)], axis=1)

In [15]:
df.drop(['Product Specification'], axis=1, inplace=True)

In [ ]:
['Country Code', 'Category', 'Name', 'Avg Price', 'Currency','Nb of Offers', 'Varumärke', 'scrape_link', 'date_scraped']

In [ ]:

df_list = [ 'Rank', 'Rating', 'Reviews', 'Description',
       'Generalspecifications', 'scrape_link', 'date_scraped',
       'Product Specification', 'Bildstorlek', 'Upplösning',
       'Bildskärmsteknik', 'HDR', 'Smart TV', 'Lanseringsår', 'Pris/tum',
       'Bakbelysning', 'Diodplacering', 'Teknologier', 'Paneltyp',
       'Produktnamn', 'Kategori', 'TV-mottagare, digital', 'Dubbla mottagare',
       'Stöd för CI/CA-modul', 'CI+', 'Typ av digital-mottagare',
       'Analog 3,5mm-ingång (Aux)', 'Digital optisk ut (S/PDIF)', 'HDMI',
       'Antal HDMI-ingångar', 'HDCP-version', 'HDMI-version',
       'Standarder som stöds', 'Hörlursutgång', 'Komponent in (YPbPr/YCbCr)',
       'Komposit in', 'USB-kontakt', 'Inspelningsfunktion via USB (PVR)',
       'USB 2.0', 'USB 3.1 Gen 1 (5Gbps)', 'USB-kontakter (totalt)', 'VGA in',
       'Övriga anslutningar', 'Bildfrekvens', 'Bildförhållande',
       'Bildstorlek (cm)', 'Ljusstyrka', 'Rörelseförbättring (frekvens/index)',
       'Skärmtyp', 'Stödda videoformat', 'Uppdateringsfrekvens',
       'Bildfiler som stöds', 'Ljudfiler som stöds', 'Ljudformat',
       'Videofiler som stöds', 'Tillgängliga färger', 'Utformning panel',
       'Djup, utan stativ', 'Höjd, utan stativ', 'Mått (BxHxD)', 'Bredd',
       'Djup', 'Höjd', 'Produktvikt', 'Vikt, utan stativ', 'Funktioner',
       'Internettjänster/appar som stöds', 'Operativsystem',
       'Ethernet-anslutning', 'Trådlöst nätverk (Wi-Fi)',
       'Trådlös nätverkstandard (IEEE 802.11)', '3D-stöd (3D-ready)', 'DLNA',
       'Full HD (HD ready 1080p)', '4K120', 'Ultra HD-certifierad (UHD/4K)',
       'Typ av HDR', 'Streaming-standarder', 'VESA-fäste', 'Typ av VESA-fäste',
       'Inbyggda högtalare', 'Processorhastighet', 'Anslutning för 9V/12V/14V',
       'Effektförbrukning drift (min-max)',
       'Effektförbrukning standby/viloläge',
       'Energiförbrukning i HDR-läge (per 1000h)',
       'Energiförbrukning i SDR-läge (per 1000h)', 'Energiklass (A-G)',
       'Energiklass (HDR)']

In [16]:
#df2 = df.reset_index().rename(columns={'index':'id'})
df_long = df.set_index(['Country Code', 'Varumärke','Category', 'Name', 'Nb of Offers', 'Lanseringsår', 'Avg Price', 'Currency', 'scrape_link', 'date_scraped']).stack().reset_index()

In [17]:
df_long

,Country Code,Varumärke,Category,Name,Nb of Offers,Lanseringsår,Avg Price,Currency,scrape_link,date_scraped,level_10,0
0,SW,LG,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",15,2021,17598.4,SEK,https://www.prisjakt.nu/produkt.php?p=5732274,12/08/2022 21:43:56,Rank,Rank 1
1,SW,LG,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",15,2021,17598.4,SEK,https://www.prisjakt.nu/produkt.php?p=5732274,12/08/2022 21:43:56,Rating,4.2
2,SW,LG,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",15,2021,17598.4,SEK,https://www.prisjakt.nu/produkt.php?p=5732274,12/08/2022 21:43:56,Reviews,(22 omdömen)
3,SW,LG,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",15,2021,17598.4,SEK,https://www.prisjakt.nu/produkt.php?p=5732274,12/08/2022 21:43:56,Description,"Det billigaste priset för LG OLED65C1 65"" 4K U..."
4,SW,LG,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",15,2021,17598.4,SEK,https://www.prisjakt.nu/produkt.php?p=5732274,12/08/2022 21:43:56,Bildstorlek,65 tum
...,...,...,...,...,...,...,...,...,...,...,...,...
79,SW,LG,TV,"LG 65QNED99 65"" 8K (7680x4320) LCD Smart TV",6,2021,40322.5,SEK,https://www.prisjakt.nu/produkt.php?p=5753940,12/08/2022 21:48:12,Bildstorlek,65 tum
80,SW,LG,TV,"LG 65QNED99 65"" 8K (7680x4320) LCD Smart TV",6,2021,40322.5,SEK,https://www.prisjakt.nu/produkt.php?p=5753940,12/08/2022 21:48:12,Upplösning,8K (7680x4320)
81,SW,LG,TV,"LG 65QNED99 65"" 8K (7680x4320) LCD Smart TV",6,2021,40322.5,SEK,https://www.prisjakt.nu/produkt.php?p=5753940,12/08/2022 21:48:12,Bildskärmsteknik,LCD
82,SW,LG,TV,"LG 65QNED99 65"" 8K (7680x4320) LCD Smart TV",6,2021,40322.5,SEK,https://www.prisjakt.nu/produkt.php?p=5753940,12/08/2022 21:48:12,HDR,Ja


In [18]:
df.head()

,Country Code,Category,Name,Avg Price,Currency,Nb of Offers,Rank,Rating,Reviews,Description,scrape_link,date_scraped,Bildstorlek,Upplösning,Bildskärmsteknik,HDR,Smart TV,Lanseringsår,Varumärke
0,SW,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",17598.400000,SEK,15,Rank 1,4.2,(22 omdömen),"Det billigaste priset för LG OLED65C1 65"" 4K U...",https://www.prisjakt.nu/produkt.php?p=5732274,12/08/2022 21:43:56,65 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,LG
1,SW,TV,"LG OLED55C1 55"" 4K Ultra HD (3840x2160) OLED S...",12901.133333,SEK,15,Rank 2,4.1,(3 omdömen),"Det billigaste priset för LG OLED55C1 55"" 4K U...",https://www.prisjakt.nu/produkt.php?p=5732268,12/08/2022 21:44:15,55 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,LG
2,SW,TV,"Philips 58PUS9006 58"" 4K Ultra HD (3840x2160) ...",9675.714286,SEK,7,Rank 3,3.7,(2 omdömen),Det billigaste priset för Philips 58PUS9006 58...,https://www.prisjakt.nu/produkt.php?p=5785188,12/08/2022 21:44:44,58 tum,4K Ultra HD (3840x2160),LCD,Ja,Ja,,Philips
3,SW,TV,"LG OLED77B1 77"" 4K Ultra HD (3840x2160) OLED S...",23323.333333,SEK,3,Rank 4,4.1,(4 omdömen),"Det billigaste priset för LG OLED77B1 77"" 4K U...",https://www.prisjakt.nu/produkt.php?p=5724842,12/08/2022 21:45:20,77 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,LG
4,SW,TV,"LG OLED55B1 55"" 4K Ultra HD (3840x2160) OLED S...",10875.500000,SEK,10,Rank 5,4.2,(5 omdömen),"Det billigaste priset för LG OLED55B1 55"" 4K U...",https://www.prisjakt.nu/produkt.php?p=5724832,12/08/2022 21:45:51,55 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,LG


In [20]:
df.to_csv("data_50_products.csv")

    prod_specifications = hun.find_all("div",{"class":"Column-sc-1jnk5ag-2 bIOAXd"})
    def get_specs(prod_specifications):
        nam = []
        val = []
        for spec in prod_specifications:
            for prop in spec.find_all("span", {"class":"Text--bzqghn wmwsE bodysmalltext PropertyName-sc-1jnk5ag-5 hEdIXF"}):
                prop_n  = prop.get_text()
                nam.append(prop_n)
            for prop in spec.find_all("span", {"class":"Text--bzqghn ddqBq bodysmalltext PropertyValue-sc-1jnk5ag-6 etcKHO"}):
                prop_v  = prop.get_text()
                val.append(prop_v)
        specs = {nam[i]: val[i] for i in range(len(nam))}
        return(specs)

In [21]:
df_long.to_csv('df_long_first_50.csv')